# 01 Import libraries

In [ ]:
import numpy as np
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Using device: cpu


In [ ]:
import pandas as pd

In [ ]:
import importlib

## Import our own modules

In [ ]:
!rm -r Learning-the-Optimal-Solution-Path
!git clone https://github.com/Cumberkid/Learning-the-Optimal-Solution-Path.git

rm: cannot remove 'Learning-the-Optimal-Solution-Path': No such file or directory
Cloning into 'Learning-the-Optimal-Solution-Path'...
remote: Enumerating objects: 1679, done.
remote: Counting objects: 100% (649/649), done.
remote: Compressing objects: 100% (290/290), done.
remote: Total 1679 (delta 492), reused 479 (delta 359), pack-reused 1030
Receiving objects: 100% (1679/1679), 13.24 MiB | 9.95 MiB/s, done.
Resolving deltas: 100% (1129/1129), done.


(Using Colab)

In [ ]:
import sys

In [ ]:
# Add the parent directory to sys.path
sys.path.append('/content/Learning-the-Optimal-Solution-Path')

In [ ]:
import lib
importlib.reload(lib)

<module 'lib' from '/content/Learning-the-Optimal-Solution-Path/lib/__init__.py'>

In [ ]:
from lib.lsp.basis_generator import scaled_shifted_legendre
from lib.lsp.basis_tf_module import Basis_TF_SGD
from lib.lsp.learn_solution_path import learn_solution_path
from lib.lsp.loss_fn_lsp import reg_unif_weighted_logit
from lib.fast_tensor_data_loader import FastTensorDataLoader

# 02 Instantiate dataset

In [ ]:
# file path for Colab. May need to change this
X_df = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/data/X_processed.csv')
y_df = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/data/y_processed.csv')

In [ ]:
X = np.array(X_df)
y = np.array(y_df).squeeze()

In [ ]:
train_X = torch.tensor(X, dtype=torch.float32)
train_y = torch.tensor(y, dtype=torch.float32)

In [ ]:
# full gradient descent uses all data points
GD_data_loader = FastTensorDataLoader(train_X, train_y, batch_size=1000, shuffle=True, )
# stochastic gradient descent uses mini-batch
SGD_data_loader = FastTensorDataLoader(train_X, train_y, batch_size=20, shuffle=True, )
# test data
test_data_loader = FastTensorDataLoader(train_X, train_y, batch_size=1000, shuffle=False, )

In [ ]:
lam_max = 1
lam_min = 0
input_dim = X.shape[1]
loss_fn = reg_unif_weighted_logit

In [ ]:
# Read the CSV file into a DataFrame
# truth = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/results/exact_soln_list.csv')
truth = pd.read_csv('exact_soln_list.csv')

# Display the DataFrame
truth

,losses,theta_0,theta_1,theta_2,theta_3,theta_4,theta_5,theta_6,theta_7,theta_8,...,theta_36,theta_37,theta_38,theta_39,theta_40,theta_41,theta_42,theta_43,theta_44,theta_45
0,0.190793,-0.373029,0.186622,0.227443,-0.023583,0.164822,0.142992,0.168018,0.026305,0.142661,...,-0.018010,0.054908,-0.005629,0.000000,0.000000,-0.031270,-0.018010,-0.139608,-0.116840,-0.061673
1,0.192130,-0.371212,0.186634,0.227586,-0.023679,0.164875,0.142262,0.167428,0.025960,0.142993,...,-0.017600,0.054199,-0.005657,0.000069,0.000098,-0.031115,-0.017600,-0.139385,-0.116402,-0.061226
2,0.193463,-0.369401,0.186645,0.227729,-0.023775,0.164927,0.141535,0.166842,0.025615,0.143323,...,-0.017191,0.053493,-0.005685,0.000137,0.000195,-0.030961,-0.017191,-0.139162,-0.115965,-0.060781
3,0.194792,-0.367595,0.186656,0.227870,-0.023870,0.164977,0.140811,0.166258,0.025271,0.143653,...,-0.016784,0.052790,-0.005713,0.000205,0.000293,-0.030808,-0.016784,-0.138939,-0.115529,-0.060337
4,0.196117,-0.365794,0.186665,0.228009,-0.023964,0.165027,0.140089,0.165677,0.024927,0.143982,...,-0.016378,0.052089,-0.005741,0.000273,0.000390,-0.030655,-0.016378,-0.138717,-0.115094,-0.059894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,0.327684,0.692472,-0.029101,-0.033382,-0.017462,-0.005831,-0.225541,-0.146453,-0.051680,0.070009,...,0.105324,-0.201982,0.000913,0.013100,0.018598,0.063129,0.105324,0.157442,0.161747,0.171301
1020,0.325866,0.693930,-0.029932,-0.034521,-0.017548,-0.006565,-0.226194,-0.148351,-0.051246,0.068693,...,0.105340,-0.202186,0.000990,0.013040,0.018537,0.063366,0.105340,0.158131,0.162149,0.171464
1021,0.324032,0.695399,-0.030769,-0.035668,-0.017634,-0.007304,-0.226853,-0.150274,-0.050809,0.067366,...,0.105357,-0.202393,0.001066,0.012980,0.018475,0.063604,0.105357,0.158825,0.162554,0.171627
1022,0.322184,0.696878,-0.031613,-0.036822,-0.017720,-0.008050,-0.227517,-0.152223,-0.050369,0.066028,...,0.105374,-0.202601,0.001143,0.012919,0.018415,0.063843,0.105374,0.159523,0.162962,0.171792


In [ ]:
selected_columns = ['theta_0', 'theta_1', 'theta_2', 'theta_3', 'theta_4',
                    'theta_5', 'theta_6', 'theta_7', 'theta_8', 'theta_9',
                    'theta_10', 'theta_11', 'theta_12', 'theta_13', 'theta_14',
                    'theta_15', 'theta_16', 'theta_17', 'theta_18', 'theta_19',
                    'theta_20', 'theta_21', 'theta_22', 'theta_23', 'theta_24',
                    'theta_25', 'theta_26', 'theta_27', 'theta_28', 'theta_29',
                    'theta_30', 'theta_31', 'theta_32', 'theta_33', 'theta_34',
                    'theta_35', 'theta_36', 'theta_37', 'theta_38', 'theta_39',
                    'theta_40', 'theta_41', 'theta_42', 'theta_43', 'theta_44',
                    'theta_45']
true_thetas = truth[selected_columns].to_numpy()
true_losses = truth['losses'].to_numpy()

# 03 Exact Gradient with Diminishing LR by Distance Diagnostic

Recall that our method runs SGD over random $\tilde λ$'s with a linear basis $\Phi(\tilde \lambda)$ of our choice. We want to approximate $\theta$ with $\Phi(\lambda)\beta$, so the objective function is $\min_\beta h(\Phi(\tilde\lambda)\beta, \tilde\lambda) = (1-\tilde\lambda) BCE(X_\text{pass}\Phi(\tilde\lambda)\beta,\ y_\text{pass}) + \tilde\lambda BCE(X_\text{fail}\Phi(\tilde\lambda)\beta,\ y_\text{fail})$. For each batch of training data set, we randomize $\tilde\lambda$. If batch size = 1, then this is equivalent to a standard SGD.

We use diminishing learning rate for better demonstrate convergence. If we use a constant learning rate, the solution path error will eventually do a random walk after descending to a certain threshold value.

Set weighted_avg to 'False' see this random walk.

In [ ]:
max_epochs = 2000

In [ ]:
phi_lam = scaled_shifted_legendre

In [ ]:
def thresh_basis(basis_dim):
    return 1e-9

## num basis func = 5

In [ ]:
basis_dim = 5
init_lr = 0.125

In [ ]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, weight, lr = learn_solution_path(input_dim, basis_dim, phi_lam, max_epochs,
                                                               GD_data_loader, test_data_loader, loss_fn,
                                                               lam_min, lam_max, true_losses, init_lr=init_lr,
                                                               diminish=True, thresh_basis=thresh_basis,
                                                               record_frequency=10, distribution='uniform',
                                                               device=device, trace_frequency=10)
sup_err_history = np.array(sup_err_history)

diminish at iteration #8, new lr = 0.12125
diminish at iteration #10, new lr = 0.1176125
--------approximate solution path for # itr = 10 complete--------
# itr: 10	 sup error: 3.808290511369705
diminish at iteration #13, new lr = 0.114084125
diminish at iteration #17, new lr = 0.11066160124999999
--------approximate solution path for # itr = 20 complete--------
# itr: 20	 sup error: 0.4882354736328125
diminish at iteration #23, new lr = 0.10734175321249999
diminish at iteration #30, new lr = 0.10412150061612499
--------approximate solution path for # itr = 30 complete--------
# itr: 30	 sup error: 0.31499093770980835
diminish at iteration #39, new lr = 0.10099785559764124
--------approximate solution path for # itr = 40 complete--------
# itr: 40	 sup error: 0.061185866594314575
--------approximate solution path for # itr = 50 complete--------
# itr: 50	 sup error: 0.040966540575027466
diminish at iteration #51, new lr = 0.097967919929712
--------approximate solution path for # itr = 

In [ ]:
file_path = 'LSP_results_exact_fixed_basis.csv'

LSP_results_exact_fixed_basis = pd.DataFrame(np.column_stack((num_itr_history, sup_err_history)), columns=['num_itr', 'sup_err_5'])

# Save the DataFrame to a CSV file
LSP_results_exact_fixed_basis.to_csv(file_path, index=False)

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df

,num_itr,sup_err_5
0,10.0,3.808291
1,20.0,0.488235
2,30.0,0.314991
3,40.0,0.061186
4,50.0,0.040967
...,...,...
195,1960.0,0.000957
196,1970.0,0.000955
197,1980.0,0.000945
198,1990.0,0.000941


## num basis func = 7

In [ ]:
basis_dim = 7
init_lr = 0.0625

In [ ]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, weight, lr = learn_solution_path(input_dim, basis_dim, phi_lam, max_epochs,
                                                               GD_data_loader, test_data_loader, loss_fn,
                                                               lam_min, lam_max, true_losses, init_lr=init_lr,
                                                               diminish=True, thresh_basis=thresh_basis,
                                                               record_frequency=10, distribution='uniform',
                                                               device=device, trace_frequency=10)
sup_err_history = np.array(sup_err_history)

--------approximate solution path for # itr = 10 complete--------
# itr: 10	 sup error: 0.20327037572860718
diminish at iteration #13, new lr = 0.060625
--------approximate solution path for # itr = 20 complete--------
# itr: 20	 sup error: 0.13757383823394775
diminish at iteration #30, new lr = 0.05880625
--------approximate solution path for # itr = 30 complete--------
# itr: 30	 sup error: 0.04141576588153839
diminish at iteration #39, new lr = 0.0570420625
--------approximate solution path for # itr = 40 complete--------
# itr: 40	 sup error: 0.017672955989837646
--------approximate solution path for # itr = 50 complete--------
# itr: 50	 sup error: 0.008210957050323542
diminish at iteration #51, new lr = 0.055330800624999996
--------approximate solution path for # itr = 60 complete--------
# itr: 60	 sup error: 0.007690101861953735
diminish at iteration #66, new lr = 0.053670876606249994
--------approximate solution path for # itr = 70 complete--------
# itr: 70	 sup error: 0.0070

In [ ]:
file_path = 'LSP_results_exact_fixed_basis.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

df['sup_err_7'] = sup_err_history

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)

## num basis func = 9

In [ ]:
basis_dim = 9
init_lr = 0.0625

In [ ]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, weight, lr = learn_solution_path(input_dim, basis_dim, phi_lam, max_epochs,
                                                               GD_data_loader, test_data_loader, loss_fn,
                                                               lam_min, lam_max, true_losses, init_lr=init_lr,
                                                               diminish=True, thresh_basis=thresh_basis,
                                                               record_frequency=10, distribution='uniform',
                                                               device=device, trace_frequency=10)
sup_err_history = np.array(sup_err_history)

diminish at iteration #4, new lr = 0.060625
--------approximate solution path for # itr = 10 complete--------
# itr: 10	 sup error: 0.8996663689613342
--------approximate solution path for # itr = 20 complete--------
# itr: 20	 sup error: 6.192219525575638
diminish at iteration #23, new lr = 0.05880625
diminish at iteration #30, new lr = 0.0570420625
--------approximate solution path for # itr = 30 complete--------
# itr: 30	 sup error: 5.673669129610062
diminish at iteration #39, new lr = 0.055330800624999996
--------approximate solution path for # itr = 40 complete--------
# itr: 40	 sup error: 3.775999575853348
--------approximate solution path for # itr = 50 complete--------
# itr: 50	 sup error: 0.6472294330596924
diminish at iteration #51, new lr = 0.053670876606249994
--------approximate solution path for # itr = 60 complete--------
# itr: 60	 sup error: 0.6392213702201843
diminish at iteration #66, new lr = 0.052060750308062495
--------approximate solution path for # itr = 70 c

In [ ]:
file_path = 'LSP_results_exact_fixed_basis.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

df['sup_err_9'] = sup_err_history

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)

# Noisy Gradient Oracle with $lr = \beta/\sqrt{t}$

Train the SGD model for our method using noisy gradient (mini-batch SGD) and record the sup error along the solution path ($\epsilon$) achieved after executing some number of gradient calls (epochs).

We use a shrinking rate $lr = \beta/\sqrt{t}$ for previously tuned $\beta$.

In [ ]:
max_epochs = 3000

In [ ]:
phi_lam = scaled_shifted_legendre

In [ ]:
# beta/sqrt(t)
def step_size(t, beta):
    return beta/np.sqrt(t+1)

In [ ]:
def thresh_basis(basis_dim):
    return 1e-9

## num basis func = 5

In [ ]:
basis_dim = 5
beta = 0.5

In [ ]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, weight, lr = learn_solution_path(input_dim, basis_dim, phi_lam, max_epochs,
                                                               SGD_data_loader, test_data_loader, loss_fn,
                                                               lam_min, lam_max, true_losses,
                                                              #  init_lr=init_lr,
                                                               step_size=step_size, const=beta, weighted_avg=True,
                                                               thresh_basis=1e-9,
                                                               record_frequency=10, distribution='uniform',
                                                               device=device, trace_frequency=100)
sup_err_history = np.array(sup_err_history)

--------approximate solution path for # itr = 100 complete--------
# itr: 100	 sup error: 0.002113640308380127
--------approximate solution path for # itr = 200 complete--------
# itr: 200	 sup error: 0.0008128881454467773
--------approximate solution path for # itr = 300 complete--------
# itr: 300	 sup error: 0.0010091662406921387
--------approximate solution path for # itr = 400 complete--------
# itr: 400	 sup error: 0.0006948709487915039
--------approximate solution path for # itr = 500 complete--------
# itr: 500	 sup error: 0.0007435381412506104
--------approximate solution path for # itr = 600 complete--------
# itr: 600	 sup error: 0.0009127557277679443
--------approximate solution path for # itr = 700 complete--------
# itr: 700	 sup error: 0.0008178055286407471
--------approximate solution path for # itr = 800 complete--------
# itr: 800	 sup error: 0.0010140836238861084
--------approximate solution path for # itr = 900 complete--------
# itr: 900	 sup error: 0.0010116696357

In [ ]:
file_path = 'LSP_results_noisy_fixed_basis.csv'

LSP_results_exact_fixed_basis = pd.DataFrame(np.column_stack((num_itr_history, sup_err_history)), columns=['num_itr', 'sup_err_5'])

# Save the DataFrame to a CSV file
LSP_results_exact_fixed_basis.to_csv(file_path, index=False)

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df

,num_itr,sup_err_5
0,10.0,0.113425
1,20.0,0.014235
2,30.0,0.006429
3,40.0,0.005368
4,50.0,0.007031
...,...,...
295,2960.0,0.000971
296,2970.0,0.000973
297,2980.0,0.000970
298,2990.0,0.000969


In [ ]:
file_path = 'LSP_results_exact_fixed_basis.csv'

df = pd.read_csv(file_path)

# Add a new column for a different basis dimension
df['sup_err_5'] = sup_err_history

# Save the updated DataFrame back to the CSV file
df.to_csv(file_path, index=False)

## num basis func = 7

In [ ]:
basis_dim = 7
beta = 0.125

In [ ]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, weight, lr = learn_solution_path(input_dim, basis_dim, phi_lam, max_epochs,
                                                               SGD_data_loader, test_data_loader, loss_fn,
                                                               lam_min, lam_max, true_losses,
                                                               step_size=step_size, const=beta, weighted_avg=True,
                                                               thresh_basis=thresh_basis,
                                                               record_frequency=10, distribution='uniform',
                                                               device=device, trace_frequency=100)
sup_err_history = np.array(sup_err_history)

--------approximate solution path for # itr = 100 complete--------
# itr: 100	 sup error: 0.0057676732540130615
--------approximate solution path for # itr = 200 complete--------
# itr: 200	 sup error: 0.0025493204593658447
--------approximate solution path for # itr = 300 complete--------
# itr: 300	 sup error: 0.001119852066040039
--------approximate solution path for # itr = 400 complete--------
# itr: 400	 sup error: 0.0006137937307357788
--------approximate solution path for # itr = 500 complete--------
# itr: 500	 sup error: 0.0008137822151184082
--------approximate solution path for # itr = 600 complete--------
# itr: 600	 sup error: 0.0004491657018661499
--------approximate solution path for # itr = 700 complete--------
# itr: 700	 sup error: 0.00023087859153747559
--------approximate solution path for # itr = 800 complete--------
# itr: 800	 sup error: 0.00036960840225219727
--------approximate solution path for # itr = 900 complete--------
# itr: 900	 sup error: 0.00021415948

In [ ]:
file_path = 'LSP_results_exact_fixed_basis.csv'

df = pd.read_csv(file_path)

# Add a new column for a different basis dimension
df['sup_err_7'] = sup_err_history

# Save the updated DataFrame back to the CSV file
df.to_csv(file_path, index=False)

## num basis func = 9

In [ ]:
basis_dim = 9
beta = 0.0625

In [ ]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, weight, lr = learn_solution_path(input_dim, basis_dim, phi_lam, max_epochs,
                                                               SGD_data_loader, test_data_loader, loss_fn,
                                                               lam_min, lam_max, true_losses,
                                                               step_size=step_size, const=beta, weighted_avg=True,
                                                               thresh_basis=thresh_basis,
                                                               record_frequency=10, distribution='uniform',
                                                               device=device, trace_frequency=100)
sup_err_history = np.array(sup_err_history)

--------approximate solution path for # itr = 100 complete--------
# itr: 100	 sup error: 0.00862722098827362
--------approximate solution path for # itr = 200 complete--------
# itr: 200	 sup error: 0.0019429028034210205
--------approximate solution path for # itr = 300 complete--------
# itr: 300	 sup error: 0.0006385147571563721
--------approximate solution path for # itr = 400 complete--------
# itr: 400	 sup error: 0.0005068778991699219
--------approximate solution path for # itr = 500 complete--------
# itr: 500	 sup error: 0.0006736516952514648
--------approximate solution path for # itr = 600 complete--------
# itr: 600	 sup error: 0.0006954818964004517
--------approximate solution path for # itr = 700 complete--------
# itr: 700	 sup error: 0.00048214197158813477
--------approximate solution path for # itr = 800 complete--------
# itr: 800	 sup error: 0.00037291646003723145
--------approximate solution path for # itr = 900 complete--------
# itr: 900	 sup error: 0.000377088785

In [ ]:
file_path = 'LSP_results_exact_fixed_basis.csv'

df = pd.read_csv(file_path)

# Add a new column for a different basis dimension
df['sup_err_9'] = sup_err_history

# Save the updated DataFrame back to the CSV file
df.to_csv(file_path, index=False)